# BigThing v2 — Portfolio Intelligence Walkthrough

This notebook walks through each module step-by-step for validation.

In [ ]:
import sys, os, logging

# Ensure src is on the path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

%load_ext autoreload
%autoreload 2

logging.basicConfig(format='%(asctime)s %(levelname)-8s %(name)s  %(message)s',
                    datefmt='%H:%M:%S', level=logging.INFO)

from bigthing.config import load_config
config = load_config('config.json')
print(f'Portfolio value: ${config.portfolio.total_value:,.0f}')
print(f'Holdings: {[h.ticker for h in config.portfolio.holdings]}')
print(f'Universe sources: {config.universe.sources}')

## Module 1: Market Regime Engine

In [ ]:
from bigthing.regime import analyze_regime

regime = analyze_regime(config.regime, config.data)

print(f'Classification: {regime.classification}')
print(f'Multiplier: {regime.multiplier}x')
print(f'SPY: ${regime.spy_price:.2f} | 200 MA: ${regime.spy_200ma:.2f} | 50 MA: ${regime.spy_50ma:.2f}')
print(f'VIX: {regime.vix_level:.1f}')
print(f'10Y Yield: {regime.treasury_yield:.2f}%')
print()
print('Signals:')
for k, v in regime.signals.items():
    print(f'  {k}: {v}')
print()
print('Explanation:')
print(regime.explanation)

## Step 2: Build Universe & Fetch Data

In [ ]:
from bigthing.universe import build_universe, PreFilterConfig
from bigthing.data_provider import DataProvider

pf = PreFilterConfig(
    min_price=config.universe.min_price,
    max_price=config.universe.max_price,
    min_avg_volume=config.universe.min_avg_volume,
    sources=config.universe.sources,
)
universe_tickers = build_universe(pf)
print(f'Universe: {len(universe_tickers)} tickers')

holding_tickers = [h.ticker for h in config.portfolio.holdings]
all_tickers = sorted(set(universe_tickers + holding_tickers + [config.regime.spy_ticker]))
print(f'Total tickers to fetch: {len(all_tickers)}')

In [ ]:
provider = DataProvider(config=config.data, batch_size=config.universe.batch_size)
market_data = provider.fetch_all(
    tickers=all_tickers,
    lookback_days=config.data.daily_lookback_days,
    fetch_fundamentals=True,
)
print(f'Daily data: {len(market_data.daily)} tickers')
print(f'Fundamentals: {len(market_data.fundamentals)} tickers')
print(f'Coverage: {market_data.coverage_pct:.1f}%')

## Module 2: Portfolio Health Engine

In [ ]:
from bigthing.health import analyze_portfolio_health

health = analyze_portfolio_health(config, market_data, regime)

print(f'Total Invested: ${health.total_invested:,.0f}')
print(f'Current Value:  ${health.total_current_value:,.0f}')
print(f'P&L: {health.total_pnl_pct:+.1f}%')
print()
for h in health.holdings:
    print(f'{h.ticker:6s}  ${h.current_price:>8.2f}  P&L: {h.unrealized_pnl_pct:>+6.1f}%  '
          f'Score: {h.total_score}/10  [{h.decision}]')
    print(f'         Trend={h.trend_score} Fund={h.fundamental_score} RS={h.relative_strength_score} Macro={h.macro_alignment_score}')
    print(f'         Stop: ${h.suggested_stop:.2f}  Risk: {h.risk_as_pct_of_portfolio:.2f}% of portfolio')
    print()

if health.actions_required:
    print('ACTIONS REQUIRED:')
    for a in health.actions_required:
        print(f'  - {a}')

## Module 3: Opportunity Scanner

In [ ]:
from bigthing.scanner import scan_opportunities

scanner = scan_opportunities(config, market_data, regime)

print(f'Scanned: {scanner.universe_scanned} stocks')
print(f'Passed filters: {scanner.passed_filter}')
print(f'Top candidates: {len(scanner.candidates)}')
print()

for i, c in enumerate(scanner.candidates, 1):
    print(f'#{i} {c.ticker} ({c.sector})')
    print(f'   Price: ${c.current_price:.2f}  Score: {c.composite_score:.0f}/100')
    print(f'   Entry: ${c.entry_zone_low:.2f} - ${c.entry_zone_high:.2f}')
    print(f'   Stop: ${c.suggested_stop:.2f}  Risk/share: ${c.risk_per_share:.2f}')
    print(f'   Position: {c.position_size_shares} shares (${c.capital_required:,.0f})')
    print(f'   RSI: {c.rsi:.1f}  Vol ratio: {c.avg_volume_ratio:.2f}x')
    print(f'   {c.bull_scenario}')
    print(f'   {c.base_scenario}')
    print(f'   {c.bear_scenario}')
    print()

## Module 4: Capital Allocation Engine

In [ ]:
from bigthing.allocator import compute_allocation

allocation = compute_allocation(config, regime, health, scanner)

print(f'Portfolio: ${allocation.total_portfolio_value:,.0f}')
print(f'Invested:  ${allocation.invested_value:,.0f} ({allocation.total_exposure_pct:.1f}%)')
print(f'Cash:      ${allocation.cash_value:,.0f} ({allocation.cash_pct:.1f}%)')
print(f'Positions: {allocation.position_count} / {allocation.max_positions}')
print()

print('Sector Concentration:')
for sector, pct in allocation.sector_concentration.items():
    print(f'  {sector}: {pct:.1f}%')

if allocation.trim_exit_plans:
    print('\nTrim/Exit Plans:')
    for p in allocation.trim_exit_plans:
        print(f'  {p.action} {p.ticker}: {p.shares} shares — {p.rationale}')

if allocation.buy_plans:
    print('\nBuy Plans:')
    for p in allocation.buy_plans:
        print(f'  BUY {p.ticker}: {p.shares} shares @ ${p.entry_price:.2f}')
        print(f'    Stop: ${p.stop_price:.2f}  Risk: ${p.risk_amount:.0f} ({p.risk_pct_of_portfolio:.2f}%)')
        print(f'    Capital: ${p.capital_required:,.0f}')

print(f'\nWeekly Plan: {allocation.weekly_deployment_plan}')
print('\nRisk Notes:')
for n in allocation.risk_notes:
    print(f'  - {n}')

## Preview Email Report

In [ ]:
from bigthing.emailer import build_report_html
from IPython.display import HTML

html = build_report_html(regime, health, scanner, allocation, config.external_holdings)
HTML(html)